1. Upload eff_output.tar.gz
2. Run all

In [ ]:
!tar -xzf eff_output.tar.gz

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as tk
import scipy.special as spi
from scipy.optimize import curve_fit

from google.colab import files
import csv

In [ ]:
params = {
   'axes.labelsize': 21,
   'font.size': 16,
   'font.family': 'sans-serif',
   'font.serif': 'Arial',
   'legend.fontsize': 18,
   'xtick.labelsize': 18,
   'ytick.labelsize': 18,
   'axes.labelpad': 15,

   'figure.figsize': [10,8], # value in inches based on dpi of monitor
   'figure.dpi': 105.5, # My monitor has a dpi of around 105.5 px/inch

   'axes.grid': True,
   'grid.linestyle': '-',
   'grid.alpha': 0.25,
   'axes.linewidth': 1,
   'figure.constrained_layout.use': True,


   # Using Paul Tol's notes:
   'axes.prop_cycle':
      mpl.cycler(color=['#4477aa', # blue
                        '#ee6677', # red/pink
                        '#228833', # green
                        '#aa3377', # purple
                        '#66ccee', # cyan
                        '#ccbb44', # yellow
                        '#bbbbbb', # grey
                        ]),

      # Pick either the cycler above, or the cycler below:

      # (mpl.cycler(color=['#4477aa', # blue
      #                     '#ee6677', # red/pink
      #                     '#228833', # green
      #                     '#aa3377', # purple
      #                     '#66ccee', # cyan
      #                     '#ccbb44', # yellow
      #                     '#bbbbbb', # grey
      #                     ]) +
      #   mpl.cycler(linestyle=['-', # solid
      #                         '--', # dashed
      #                         ':', # dotted
      #                         '-.', # dash dot
      #                         (0, (3, 1, 1, 1, 1, 1)), # narrow dash dot dot
      #                         (0, (1, 2, 5, 2, 5, 2)), # dash dash dot
      #                         (0, (5, 2.5, 1, 2.5, 1, 2.5)), # dash dot dot
      #                         ])),

   'lines.linewidth': 2.5,

   'image.cmap': 'jet',
}


plt.rcParams.update(params)

In [ ]:
def gaussian(x,mu,sigma):
  return np.exp(-0.5*((x-mu)/sigma)**2)/(np.sqrt(2*np.pi)*sigma)

In [ ]:
efficiencies = []
num=0
for i in range(1,5001,1):
  try:
    infile = open("eff_590_0/" + str(i) + ".csv","r")
  except FileNotFoundError:
    continue
  all = infile.readlines()
  if int(all[0])!=0:
    efficiencies.append(int(all[0])/int(all[1]))
    num+=1
print(efficiencies)
print(num)

In [ ]:
n_bins = 70

#set up bins
bins = np.linspace(min(efficiencies),max(efficiencies), n_bins+1)
bin_width = (max(efficiencies)-min(efficiencies))/n_bins
bin_centres = []
for i in range(len(bins)-1):
  bin_centres.append((bins[1]-bins[0])*(i)+(bins[1]+bins[0])/2)

entries, bc, p=plt.hist(efficiencies, bins=bins, density=True, weights = np.ones_like(efficiencies)/num)
plt.xlabel("Efficiency")
plt.ylabel("Normalised counts")
plt.title("[Pure CF4, 60 Torr, dV = 590V]")


#Fit, plot Gaussian
parameters, cov_matrix = curve_fit(gaussian, bin_centres, entries, p0=[0.175,0.001])
y=[]
for i in bin_centres:
  y.append(gaussian(i,parameters[0],parameters[1]))
plt.plot(bin_centres,y,"o")
plt.legend(["Fitted Gaussian distribution","Simulation"])
print("mean: "+str(parameters[0]))
print("SD: "+str(parameters[1]))
print(np.divide(np.sqrt(np.diag(cov_matrix)),np.sqrt(num)))